In [1]:
from trendsAPI import TrendReq # API non officielle
import json
import pandas as pd
from datetime import datetime, timedelta
import re
import sys

import sys

sys.argv[0] (nom du fichier)

sys.argv[1] (1er paramètre, etc...)

Documentation de l'API :

https://github.com/GeneralMills/pytrends

### open issues

Limites de requêtes sur Google :
- générer un string aléatoire à passer en paramètre custom_useragent
https://github.com/GeneralMills/pytrends/issues/77
- si erreur, utiliser un autre compte gmail

Format JSON :
- quel format lu par chart.js, d3.js ?
http://stackoverflow.com/questions/24929931/drawing-line-chart-in-chart-js-with-json-response

Plus de 5 recherches à la fois :
- cf open issues sur pytrends
https://github.com/GeneralMills/pytrends/issues/77

[résolu] Gérer les timestamps et timezones :
- cf : https://docs.python.org/3/library/datetime.html#datetime.tzinfo

## Les tendances sur les recherches

- 'Élection présidentielle française de 2017' : '/m/0n49cn3' (Élection)

#### Codes associés aux candidats :
- Manuel Valls : '/m/047drb0' (Ancien Premier ministre français)
- François Fillon : '/m/0fqmlm' (Ancien Ministre de l’Écologie, du Développement durable et de l’Énergie)
- Vincent Peillon : '/m/0551vp' (Homme politique)
- François Bayrou : '/m/02y2cb' (Homme politique)
- François Hollande : '/m/02qg4z' (Président de la République française)
- Jean-Luc Mélanchon : '/m/04zzm99' (Homme politique)
- Yannick Jadot : '/m/05zztc0' (Ecologiste)
- Nicolas Dupont-Aignan (Debout la France) : '/m/0f6b18'
- Michèle Alliot-Marie (Indépendante) : '/m/061czc' (Ancienne Ministre de l'Intérieur de France)
- Nathalie Artaud (LO) : pas dispo
- Philippe Poutou (NPA) : '/m/0gxyxxy'
- Emmanuel Macron : '/m/011ncr8c' (Ancien Ministre de l'Économie et des Finances)
- Jacques Cheminade : '/m/047fzn'
- Marine Le Pen : '/m/02rdgs'

#### Codes associés aux partis :
- LR : '/g/11b7n_r2jq'
- PS : '/m/01qdcv'
- FN : '/m/0hp7g'
- EELV : '/m/0h7nzzw'
- FI (France Insoumise) : pas dispo direttement... utiliser le terme de recherche : "france insoumise"
- PCF : '/m/01v8x4'
- Debout la France : '/m/02rwc3q'
- MoDem : '/m/02qt5xp' (Mouvement Démocrate)
- Lutte Ouvrière : '/m/01vvcv'
- Nouveau Parti Anticapitalise : '/m/04glk_t'
- En marche! : pas dispo

#### Codes couleurs :

- dark blue: #637b85
- green: #2c9c69
- yellow: #dbba34
- red: #c62f29


In [4]:
# Formattage de la date pour gérer le format heure:minute
def convert_date_column(dataframe, out='%d/%m %H:%M'):
    dates = []
    if 'PST' in dataframe['Date'][0]: # format de date anglais
        rdict = {',': '', ' PST': ''}
        in_format = '%b %d %Y %H:%M' # ex: Jan 18 2017 12:00
        out_format = out # ex: 18/01 12:00
    elif 'UTC−8' in dataframe['Date'][0]: # format de date français
        rdict = {' à': '', ' UTC−8': '', 'janv.': '01', 'févr.': '02', 'mars': '03', 'avr.': '04', 
                 'mai': '05', 'juin': '06', 'juil.': '07', 'août': '08', 'sept.': '09', 'oct.': '10', 
                 'nov.': '11', 'déc.': '12'}
        in_format = '%d %m %Y %H:%M' # ex: 18 01 2017 12:00
        out_format = out # ex: 18/01 12:00
    
    else: # si les dates ne contiennent pas l'heure, 
        return
        
    robj = re.compile('|'.join(rdict.keys()))
    for date in dataframe['Date']: # Conversion des dates
        t = datetime.strptime(robj.sub(lambda m: rdict[m.group(0)], date), in_format) + timedelta(hours=9) # GMT+1
        dates.append(datetime.strftime(t, out_format))
        
    dataframe['Date'] = dates
    return

#### Fonction qui sauvegarde les requetes via l'API en JSON

In [51]:
def trends_to_json(query='candidats_majeurs', periode='3d', geo='FR'):

    # Formats possibles pour la date : now 1-H, now 4-H, now 1-d, now 7-d, today 1-m, today 3-m
    periodes = {'1h': 'now 1-H', '4h': 'now 4-H', '1d': 'now 1-d', '3d': 'now 3-d',
                '7d': 'now 7-d', '1m': 'today 1-m', '3m': 'today 3-m'}

    # Les termes de recherche (5 au maximum separes par des virgules)
    # On associe a un type de recherche la liste des parametres correspondants
    queries = {'candidats_majeurs': '/m/047drb0, /m/05zztc0, /m/02rdgs, /m/011ncr8c, /m/0fqmlm',
               'partis_majeurs': '/g/11b7n_r2jq, /m/01qdcv, /m/0hp7g, /m/0h7nzzw',
              'divers_gauche': 'france insoumise, /m/01vvcv, /m/04glk_t, /m/01v8x4'} 
    # se referer a la table de correspondance ci-dessus
    
    if (query not in queries) or (periode not in periodes):
        print('Erreur de parametre')
        return
    
    users = ['pfrlepoint@gmail.com', 'pfrlepoint2@gmail.com']
    for user in users:
        try:
            # Connection to Google (use a valid Google account name for increased query limits)
            pytrend = TrendReq(user, 'projet_fil_rouge', custom_useragent='pfr')

            # Possibilite de periode personnalise : specifier deux dates (ex : 2015-01-01 2015-12-31)

            # geographie : FR (toute France), FR-A ou B ou C... (region de France par ordre alphabetique)
            # categorie politique : cat = 396

            # On fait la requete sur Google avec les parametres choisis
            payload = {'q': queries[query], 'geo': geo, 'date': periodes[periode]}
            df = pytrend.trend(payload, return_type='dataframe')

            if periode[-1] != 'm':
                convert_date_column(df, out='%d/%m %H:%M') # converts into a standard date format like: 18/01 12:00
    #         else:
    #             df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
    #             dates = []
    #             for elem in df['Date']:
    #                 dates.append(datetime.strftime(elem, '%d/%m'))
    #             df['Date'] = dates

            df.set_index('Date', inplace=True)

            # reduction du nombre de lignes du dataframe a une trentaine de points
            # pour la lisibilité du graph
            n = {'1h': 2, '4h': 1, '1d': 3, '3d': 2, '7d': 6, '1m': 1, '3m': 3}

            # Sauvegarde en JSON
            print((df.shape[0] - 1) % n[periode])
            df[(df.shape[0] - 1) % n[periode]::n[periode]].to_json(
                query + '_' + periode + '.json', orient='split')
            print('sauvegarde dans : ' + query + '_' + periode + '.json')
            return
            
        except:
            continue
    
    print('Erreur lors de la récupération des données.')
    return

#### On lance la fonction

In [52]:
# passage des arguments via sys.argv
trends_to_json(query='candidats_majeurs', periode='3d')

4
sauvegarde dans : candidats_7d.json


In [53]:
df_json = pd.read_json('candidats_7d.json', orient='split')
# print('PST' in df_json['Date'])
print(df_json)

             François Fillon  Manuel Valls  Marine Le Pen
11/01 21:00                5             2              2
12/01 03:00                7             4              3
12/01 09:00                7             2              2
12/01 15:00                6             2              1
12/01 21:00                6            19              5
13/01 03:00                4             3             10
13/01 09:00                5             4              3
13/01 15:00                5             3              2
13/01 21:00                4             3              3
14/01 03:00                3             2              4
14/01 09:00                4             2              2
14/01 15:00                5             2              2
14/01 21:00                5             3              3
15/01 03:00                4             4              3
15/01 09:00                5             5              2
15/01 15:00                4             3              2
15/01 21:00   

In [32]:
payload = {'q': '/m/047drb0, /m/0fqmlm', 'geo': 'FR',
           'date': 'today 1-m'}
pytrend = TrendReq('pfrlepoint@gmail.com', 'projet_fil_rouge', custom_useragent='pfr')
df = pytrend.trend(payload, return_type='dataframe')
df

,Date,François Fillon,Manuel Valls
0,dimanche 18 décembre 2016,33,19
1,lundi 19 décembre 2016,36,17
2,mardi 20 décembre 2016,35,16
3,mercredi 21 décembre 2016,33,100
4,jeudi 22 décembre 2016,29,52
5,vendredi 23 décembre 2016,32,28
6,samedi 24 décembre 2016,28,30
7,dimanche 25 décembre 2016,36,15
8,lundi 26 décembre 2016,33,19
9,mardi 27 décembre 2016,37,13


In [33]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
dates = []
for elem in df['Date']:
    dates.append(datetime.strftime(elem, '%d/%m'))
df['Date'] = dates

ValueError: Unknown string format

In [28]:
dates

0    2016-12-18
1    2016-12-19
2    2016-12-20
3    2016-12-21
4    2016-12-22
5    2016-12-23
6    2016-12-24
7    2016-12-25
8    2016-12-26
9    2016-12-27
10   2016-12-28
11   2016-12-29
12   2016-12-30
13   2016-12-31
14   2017-01-01
15   2017-01-02
16   2017-01-03
17   2017-01-04
18   2017-01-05
19   2017-01-06
20   2017-01-07
21   2017-01-08
22   2017-01-09
23   2017-01-10
24   2017-01-11
25   2017-01-12
26   2017-01-13
27   2017-01-14
Name: Date, dtype: datetime64[ns]